## Working with files

Reading from and writing to files is pretty simple in python when using the built in function open(). The first argument of this function is the target file, which is either the one you're try to write to, create or read from. The second argument is which "mode" you want to access the file. help(open) is fine way to get fast overview of the different modes.

### Example 
Let's quickly create a file with the open() function, write something in it and read that.

In [3]:
file_write = open("example.txt","a")
file_write.write("Hello from file!")
file_write.close

<function TextIOWrapper.close()>

In [4]:
file_read = open("example.txt","r")
message = file_read.read()
print(message)
file_read.close

<function TextIOWrapper.close()>

In these two code snippets i close the file objects manually, which is important in order to avoid resource leaks and other unwanted shenanigans. If it should have been completely by the book, the reading and writing of the file should have been in a try block, and the file closing should have been in a finally block, in order to make sure that the file would be closed even if exceptions were raised during execution or something else interrupted the program flow.
But that is a little bit cumbersome to write and python luckily has a pretty snazzy way of handling that! 

### Context managers: Managing resources
Using the "with" statement when accessing the file you be instantiate a "context manager" that will manage closing the file after you're done with it.
#### Example:
Reading file with context manager

In [5]:
# The "with" keyword is how we tell the interpreter that we want to open the file with context manager
with open("example.txt","r") as managed_file_read:
    #this is the block where we do what we want with the file's content. after this is done executing the context manager will
    #close this file for us
    message = managed_file_read.read()
    print(message)

### Under the hood:
How  does it work? First the exspression is evaluated, where it get's checked whether it implements the \_\_enter\_\_ and \_\_exit\_\_ protcols, if it does the evaluation will result in a context manager object. This object's \_\_enter\_\_ method is the called, which should return the resource, in this case a file object, after which the content of the code block is executed. If the execution of that is interrupted or completed the context manager's \_\_exit\_\_ will the be called.
This means it's possible to create our own custom context managers
#### Example:

In [6]:
from contextlib import ContextDecorator

class custom_manager(ContextDecorator):
     
    def __init__(self, file_path,mode):
        self.__path = file_path
        self.__mode = mode
        self.__file_object = None
    
    def __enter__(self):
        print("Printed from __enter__()")
        self.__file_object = open(self.__path, self.__mode)
        return  self.__file_object

    #the extra arguments in thhe __exit__ protocol is used to pass eventual exceptions thrown during execution to the __exit__
    # method.
    def __exit__(self, type, val, tb):
        print("Printed from __exit__()")
        self.__file_object.close()
    

with custom_manager("example.txt","r") as managed_file_read:
    message = managed_file_read.read()
    print(message)

Printed from __enter__()

Printed from __exit__()


This example is little bit silly, basicly tacking on some prints to the file object's functions. Connecting to a database where you need a lot of set up in order to get a working connection would make more sense.

### Context manager by decoration:
That was quite a bit of code to just, coding an entire class,  make a custom context manager. Luckily there's an easier way make one with the module contextlib!
### Example:

In [7]:
from contextlib import contextmanager

@contextmanager
def decorated_manager(file_name,mode):
    #Setup
    file= open(file_name,mode)
    print("Printed from the try-block")
    #Returning the resource
    try:
        yield file
    
    #Clean up
    finally:
        print("Printed from finally block")
        file.close()

with decorated_manager("example.txt","r") as managed_file_read:
    message = managed_file_read.read()
    print(message)

Printed from the try-block

Printed from finally block


Here we get a context manager by putting everything we did in the \_\_enter\_\_ method, except for retuning the resource, before the try block, yielding the resource is the try block and then putting the clean up tasks from the \_\_exit\_\_ method in finally block, and the decorating the function with @contextmanager

## Reading and writing Json to and from files:
Python has native support for reading and writing Json objects to and from files. By importing the json module you gain access to functionality to "dump" or "load" json.

In [8]:
import json
#Data,a dict to be exact, to be saved as a json object in a file
album= {
    "album":{
        "name":"Home Dead(EP)",
        "artist":"Kashmir",
        "tracks":["Undisturbed","Home Dead", "The Ghots Of No One","Miss You", "Just A Phase", "Mom In Love, Daddy In Space"],
        "year of release": 2001
        
    }
}

with open("album_file.json","w") as write_file:
    json.dump(album,write_file)
    print(json.dumps(album))


{"album": {"name": "Home Dead(EP)", "artist": "Kashmir", "tracks": ["Undisturbed", "Home Dead", "The Ghots Of No One", "Miss You", "Just A Phase", "Mom In Love, Daddy In Space"], "year of release": 2001}}


#### Serializing:
The json.dump method serializes the album dict into a json object, and saves it to the file passed as the second argument. The Json.dumps just serializes the dict and returns it as string representation of the Json object.
#### Deserializing:
The json.load method deserializes the json object from the fil into a python dictionary, meaning that dictionary methods are callable on the return value of the method. the json.loads method deserializes a json formatted string into the corresponding pyton object
#### Example:

In [9]:
with open("album_file.json","r") as read_file:
    data = json.load(read_file)
    #Using the dict method to extract the artist value from the nested dictionary
    print(data.get("album").get("artist"))

Kashmir
